In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib.pyplot as plt 

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
d = pd.read_csv('../input/suicide-rates-overview-1985-to-2016/master.csv')

In [ ]:
df = d.copy()

# We want to answer this question's visualizations(any matplotlib or seaborn method)
- Global Suicides(per 100K)-trend over time 1985-2016
- Global Suicides(per 100K) by Continent
- Global Suicides(per 100k) by Gender and trend over time 1985-2016
- Population-gdp_per_capita Plot
- Correlation between GDP(per Capita) and suicides per 100k
- Generation hue Gender Counter
- Which age of people suicide a most
- Which generation of people suicide a most

In [ ]:
df.head()

In [ ]:
df.tail()

# Explanation of Features
- Country : Name of the country.
- year : year of suicide (1985 - 2016)
- sex : gender (male / female)
- age : 15-24 years/25-34 years/35-54 years/5-14 years/55-74 years/75+ years
- suicides_no : the raw number of reported suicides
- population : country population
- suicides_100k : suicides_no * 100k / population
- gdp_for_year : total monetary or market value of all final goods and services produced in a country in a given year
- gdp_per_capita : the ratio of real GDP to the average population of a specific year
- generation : Boomers / G.I.Generation / Generation X / Generation Y / Millenials / Silent

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df = df.rename(columns={'country':'Country' , 'year':'Year' , 'sex':'Sex' , 'age':'Age' ,
                   'suicides_no':'SuicidesNo' , 'population':'Population' , 
                   'suicides/100k pop':'Suicides/100kPop' , 'country-year':'CountryYear' ,
                   'HDI for year':'HDIForYear' , ' gdp_for_year ($) ':'GdpForYear($)', 
                   'gdp_per_capita ($)':'GdpPerCapita','generation':'Generation'})

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(['HDIForYear','CountryYear'],axis=1,inplace=True)

In [ ]:
df.info()

# Global Suicides(per 100K)-trend over time 1985-2016

In [ ]:
display(df['Suicides/100kPop'].max())
display(df['Suicides/100kPop'].min())

In [ ]:
unique_year = df['Year'].nunique()
unique_year

In [ ]:
group_year = df.groupby('Year').sum()
group_year

In [ ]:
year_suicides = group_year['Suicides/100kPop']
year_suicides.plot()

In [ ]:
plt.subplots(figsize=(18,6))
group_year['Suicides/100kPop'].plot.bar(rot=0)
plt.title("Suicide Rates by Year");

# Global Suicides(per 100K) by Continent

In [ ]:
unique_country = df['Country'].nunique()
unique_country

In [ ]:
group_country = df.groupby('Country').sum()
group_country

In [ ]:
df_continent2 = pd.read_csv('../input/conti2/continents2.csv')
df_continent2

In [ ]:
def find_continent(df1,df2):
    for i in df1.unique():
        if i in df2.unique():
            continue
        else:
            print('MISMATCH',i)
find_continent(df.Country,df_continent2.name)

In [ ]:
df_continent2.loc[df_continent2.name.str.contains("Bosnia"),'name']='Bosnia and Herzegovina'
df_continent2.loc[df_continent2.name.str.contains("Maca"),'name']='Macau'
df_continent2.loc[df_continent2.name.str.contains("Korea, Republic of"),'name']='Republic of Korea'
df_continent2.loc[df_continent2.name.str.contains("Russia"),'name']='Russian Federation'
df_continent2.loc[df_continent2.name.str.contains("Vincent"),'name']='Saint Vincent and Grenadines'
find_continent(df.Country, df_continent2.name)

In [ ]:
df3 = df.merge(df_continent2[['name','region']], left_on='Country', right_on='name')
df3

In [ ]:
print('Unique Values of Features:\n')
for i in ['region','Year']:
    print(f'{i}:\n{sorted(df3[i].unique())}\n')

In [ ]:
df3.drop(['name'],axis=1,inplace=True)

In [ ]:
df3

In [ ]:
group_region = df3.groupby('region').sum()
group_region

In [ ]:
grf2 = group_region['Suicides/100kPop'].sort_values(ascending=False)
grf2.plot()

In [ ]:
plt.subplots(figsize=(12,6))
group_region['Suicides/100kPop'].sort_values(ascending=False).plot.bar(rot=0)
plt.title("Suicide Rates by Regions");

# Global Suicides(per 100k) by Gender and trend over time 1985-2016

In [ ]:
plt.figure(figsize=(18,5))
sns.barplot(x = "Year", y = "Suicides/100kPop", hue = "Sex",data = df3.groupby(["Year","Sex"]).sum().reset_index()).set_title("Gender vs Suicides")
# plt.xticks(rotation = 90)

# Population-gdp_per_capita Plot

In [ ]:
def split_gdp(gdp_datas):
    if 251<= gdp_datas <= 12000:
        return "251-12000"
    elif 12001<= gdp_datas <= 24000:
        return "12001-24000"
    elif 24001<= gdp_datas <= 36000:
        return "24001-36000"
    elif 36001<= gdp_datas <= 48000:
        return "36001-48000"
    elif 48001<= gdp_datas <= 60000:
        return "48001-60000"
    elif 60001<= gdp_datas <= 72000:
        return "60001-72000"
    elif 72001<= gdp_datas <= 84000:
        return "72001-84000"
    elif 84001<= gdp_datas <= 96000:
        return "84001-96000"
    elif 96001<= gdp_datas <= 108000:
        return "96001-108000"    
    else:
        return "108001-126352"
df3.GdpPerCapita = df3.GdpPerCapita.apply(split_gdp)

In [ ]:
plt.figure(figsize=(17,5))
sns.barplot(x = "GdpPerCapita", y = "Population", hue = "GdpPerCapita",data = df3.groupby(["GdpPerCapita"]).sum().reset_index()).set_title("Population vs GdpPerCapita")
# plt.xticks(rotation = 90)

# Correlation between GDP(per Capita) and suicides per 100k

In [ ]:
plt.subplots(figsize=(8,6))
sns.heatmap(df.corr(), annot = True);

- GdpPerCapita ile Suicides arasindaki korelasyon 0,0018 yani, yok sayilabilecek kadar az.

# Generation hue Gender Counter

In [ ]:
plt.figure(figsize=(17,5))
sns.barplot(x = "Generation", y = "Suicides/100kPop", hue = "Sex",data = df3.groupby(["Generation","Sex"]).sum().reset_index()).set_title("Gender vs Suicides")
# plt.xticks(rotation = 90)

# Which age of people suicide a most

In [ ]:
plt.figure(figsize=(17,5))
sns.barplot(x = "Age", y = "Suicides/100kPop", data = df3.groupby(["Age"]).sum().reset_index()).set_title("Which age of people suicide a most?")
# plt.xticks(rotation = 90)

# Which generation of people suicide a most

In [ ]:
plt.figure(figsize=(17,5))
sns.barplot(x = "Generation", y = "Suicides/100kPop", data = df3.groupby(["Generation"]).sum().reset_index()).set_title("Which age of people suicide a most?")
# plt.xticks(rotation = 90)